In [ ]:
import pandas as pd
import spacy
from scipy import spatial
import requests
import os 
from os.path import getsize
import numpy as np
import cv2
import pytesseract
import tempfile
from pdf2image import convert_from_path, convert_from_bytes
import PyPDF2
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from spacy.matcher import PhraseMatcher
import re
import string
import pickle
from PIL import Image
nlp = spacy.load('en_core_web_lg')

In [ ]:
#Function to check if the token is a noise or not  
def isNoise(token):  
    noisy_pos_tags = ['PROP']
    min_token_length = 2

    is_noise = False
    if token.pos_ in noisy_pos_tags:
        is_noise = True 
    elif token.is_stop == True:
        is_noise = True
    elif len(token.string) <= min_token_length:
        is_noise = True
    return is_noise 

In [ ]:
def cleanup(token, lower = True):
    if lower:
        token = token.lower()
    return token.strip()

In [ ]:
def clean_data(nlp_document, Is_lower_true=True):
    cleaned_list = [cleanup(word.string,Is_lower_true) for word in nlp_document if not isNoise(word)]
    return ' '.join(cleaned_list)

In [ ]:
def get_initials(fullname):
  xs = (fullname)
  name_list = xs.split()

  initials = ""

  for name in name_list:  # go through each name
    initials += name[0].upper()  # append the initial

  return initials

In [ ]:
df = pd.read_excel("./ec list.xlsx")
# df.to_excel("./ec list.xlsx")

In [ ]:
df.head()

In [ ]:
df_san = pd.read_csv('./ece_questions.csv')

In [ ]:
df_san.head()

In [ ]:
df_san = df_san.drop(labels = 'Unnamed: 0', axis =1)

In [ ]:
product = list(df_san.columns)

In [ ]:
len(product)

In [ ]:
df_new = df_san.T

In [ ]:
df_new['id'] = list(range(37))

In [ ]:
df_new

In [ ]:
Y = list(df_new['id'])

In [ ]:
list(df_new.iloc[0,:])

In [ ]:
X = []
for i in range(df_new.shape[0]):
    X.append(list(df_new.iloc[i,:-1]))

In [ ]:
X[0]

In [ ]:
dataframe = pd.DataFrame(data=X[0],columns=['quesions'])

In [ ]:
dataframe

In [ ]:
df_list=[]
for i in range(len(X)):
    df_list.append(pd.DataFrame(data=X[i],columns=['questions']))

In [ ]:
for i in range(len(df_list)):
    df_list[i]['id']=product[i]

In [ ]:
df_list[8]

In [ ]:
df_concat = pd.concat(df_list)

In [ ]:
df_concat.shape

In [ ]:
df_san.to_csv('./ece_questions_new.csv')

In [ ]:
ece_ques= df_san.to_dict()

In [ ]:
keys = list(ece_ques.keys())

In [ ]:
values = list(ece_ques.values())

In [ ]:
values[0]

In [ ]:
X = df['CLEAN_DATA']

In [ ]:
y = df['DISCIPLINE']

In [ ]:
y 

In [ ]:
for i in range(len(y)):
    y[i] = y[i].upper().strip()

In [ ]:
y

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [ ]:
y = df1['COLLEGE ']


In [ ]:
for i in y:
    print(type(i))

In [ ]:
# Create a new dataframe with two columns
df1 = df_concat[['questions', 'id']].copy()

# Remove missing values (NaN)
df1 = df1[pd.notnull(df1['questions'])]

# Renaming second column for a simpler name
df1.columns = ['X', 'Y'] 

df1.shape

In [ ]:
df1.shape

In [ ]:
pd.DataFrame(df1.Y.unique()).values

In [ ]:
df2 = df1

In [ ]:
df2.columns = [ 'Consumer_complaint','Product'] 

In [ ]:
to_clean = list(df2['Consumer_complaint'])

In [ ]:
to_clean = [clean_data(nlp(f)) for f in to_clean]

In [ ]:
df2['Consumer_complaint'] = to_clean

In [ ]:
df2['category_id'] = df2['Product'].factorize()[0]

In [ ]:
df2['category_id']

In [ ]:
category_id_df = df2[['Product', 'category_id']].drop_duplicates()

In [ ]:
category_id_df.values

In [ ]:
is_11 = df2['category_id']!=11 
is_12 = df2['category_id']!=12
is_9 = df2['category_id']!=9


In [ ]:
for i in a:
    print(i)

In [ ]:
df2 = df2[is_9]
df2 = df2[is_11]
df2 = df2[is_12]

In [ ]:
df2.shape

In [ ]:
# Dictionaries for future use
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Product']].values)

In [ ]:
df2.shape

In [ ]:
fig = plt.figure(figsize=(8,6))
colors = ['grey','grey','grey','grey','grey','grey','grey','grey','grey',
    'grey','darkblue','darkblue','darkblue']
df2.groupby('Product').Consumer_complaint.count().sort_values().plot.barh(
    ylim=0, color=colors, title= 'NUMBER OF COMPLAINTS IN EACH PRODUCT CATEGORY\n')
plt.xlabel('Number of ocurrences', fontsize = 10);

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words='english')

In [ ]:
features = tfidf.fit_transform(df2.Consumer_complaint).toarray()


In [ ]:
features.shape

In [ ]:
labels = df2.category_id

print("Each of the %d complaints is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

In [ ]:
# Finding the three most correlated terms with each of the product categories
N = 7
for Product, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("\n==> %s:" %(Product))
  print("  * Most Correlated Unigrams are: %s" %(', '.join(unigrams[-N:])))
  print("  * Most Correlated Bigrams are: %s" %(', '.join(bigrams[-N:])))

In [ ]:
X = df2['Consumer_complaint'] # Collection of documents
y = df2['Product'] # Target or the labels we want to predict (i.e., the 13 different complaints of products)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 0)

In [ ]:
# models = [
#     RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
#     LinearSVC(),
#     MultinomialNB(),
#     LogisticRegression(random_state=0)]

models = [
    LinearSVC(),
    MultinomialNB()]

# 5 Cross-validation
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
print(cv_df.head(5))

In [ ]:
cv_df

In [ ]:
mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc


In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(x='model_name', y='accuracy', 
            data=cv_df, 
            color='lightblue', 
            showmeans=True)
plt.title("MEAN ACCURACY (cv = 5)\n", size=14);

In [ ]:
X_train, X_test, y_train, y_test,indices_train,indices_test = train_test_split(features, 
                                                               labels, 
                                                               df2.index, test_size=0.25, 
                                                               random_state=1)
model = LinearSVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
# Classification report
print('\t\t\t\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_test, y_pred, 
                                    target_names=['ap','cn', 'coa','cp','dsa','dsy', 'opp', 'idbms','ota','daa','os']))

In [ ]:
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(8,8))
sns.heatmap(conf_mat, annot=True, cmap="Blues", fmt='d',
            xticklabels=category_id_df.Product.values, 
            yticklabels=category_id_df.Product.values)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title("CONFUSION MATRIX - LinearSVC\n", size=16);

In [ ]:
for predicted in category_id_df.category_id:
  for actual in category_id_df.category_id:
    if predicted != actual and conf_mat[actual, predicted] >= 20:
      print("'{}' predicted as '{}' : {} examples.".format(id_to_category[actual], 
                                                           id_to_category[predicted], 
                                                           conf_mat[actual, predicted]))
    
      display(df2.loc[indices_test[(y_test == actual) & (y_pred == predicted)]][['Product', 
                                                                'Consumer_complaint']])
      print('')

In [ ]:
model.fit(features, labels)

N = 4
for Product, category_id in sorted(category_to_id.items()):
  indices = np.argsort(model.coef_[category_id])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 1][:N]
  bigrams = [v for v in reversed(feature_names) if len(v.split(' ')) == 2][:N]
  print("\n==> '{}':".format(Product))
  print("  * Top unigrams: %s" %(', '.join(unigrams)))
  print("  * Top bigrams: %s" %(', '.join(bigrams)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 0)

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1, 2), 
                        stop_words='english')

fitted_vectorizer = tfidf.fit(X_train)
tfidf_vectorizer_vectors = fitted_vectorizer.transform(X_train)

model = LinearSVC().fit(tfidf_vectorizer_vectors, y_train)
